# Test Load Daily

Start by loading one "new" tile.

## Imports

In [1]:
import os
from glob import glob
from pytz import utc
import numpy as np
from astropy import __version__ as astropy_version
from astropy.io import fits
from astropy.table import Table, Column, join
from astropy.time import Time
from sqlalchemy import __version__ as sqlalchemy_version
from desiutil import __version__ as desiutil_version
from desiutil.names import radec_to_desiname
from desiutil.log import get_logger, DEBUG
from desispec.io.photo import gather_tractorphot, gather_targetphot
from desispec.io.meta import findfile
from desispec.scripts.zcatalog import read_redrock
from desispec.zcatalog import find_primary_spectra
from desitarget.targets import decode_targetid
from specprodDB import __version__ as specprodDB_version
import specprodDB.load as db
from specprodDB.util import cameraid, spgrpid, surveyid

## Initial Values

In [2]:
specprod = os.environ['SPECPROD']
tileid = 3867
survey, program = 'main', 'dark'
overwrite = True

## Initialize Database

In [3]:
db.log = get_logger(DEBUG)
postgresql = db.setup_db(schema='daily', hostname='db-loadbalancer.bweaver.development.svc.spin.nersc.org', username='desi_admin', overwrite=overwrite)
if overwrite:
    versions = [db.Version(package='specprod-db', version=specprodDB_version),
                db.Version(package='lsdr9-photometry', version='main'),
                db.Version(package='redshift', version='daily/v0'),
                db.Version(package='tiles', version='trunk'),
                db.Version(package='specprod', version=specprod),
                db.Version(package='numpy', version=np.__version__),
                db.Version(package='astropy', version=astropy_version),
                db.Version(package='sqlalchemy', version=sqlalchemy_version),
                db.Version(package='desiutil', version=desiutil_version)]
    db.dbSession.add_all(versions)
    db.dbSession.commit()

INFO:load.py:1415:setup_db: Begin creating tables.
INFO:load.py:1418:setup_db: Finished creating tables.


## Read tiles file

In [4]:
specprod_dir = os.path.join(os.environ['DESI_SPECTRO_REDUX'], specprod)
tiles_file = findfile('tiles').replace('.fits', '.csv')
tiles_table = Table.read(tiles_file, format='ascii.csv')
tiles_table

TILEID,SURVEY,PROGRAM,FAPRGRM,FAFLAVOR,NEXP,EXPTIME,TILERA,TILEDEC,EFFTIME_ETC,EFFTIME_SPEC,EFFTIME_GFA,GOALTIME,OBSSTATUS,LRG_EFFTIME_DARK,ELG_EFFTIME_DARK,BGS_EFFTIME_BRIGHT,LYA_EFFTIME_DARK,GOALTYPE,MINTFRAC,LASTNIGHT
int64,str7,str6,str16,str19,int64,float64,float64,float64,float64,float64,float64,float64,str8,float64,float64,float64,float64,str7,float64,int64
70004,unknown,--,unknown,unknown,4,3600.0,116.0,20.7,0.0,3619.8,0.0,1000.0,obsend,3470.8,3619.8,3784.0,3056.6,unknown,0.9,20200219
70508,unknown,--,unknown,unknown,6,1800.0,133.4125,11.6818,0.0,108.5,0.0,1000.0,obsstart,93.6,108.5,95.7,50.6,unknown,0.9,20200225
70506,unknown,--,unknown,unknown,3,900.0,133.4125,11.6818,0.0,61.6,0.0,1000.0,obsstart,53.6,61.6,54.9,33.7,unknown,0.9,20200225
70512,unknown,--,unknown,unknown,7,3150.0,132.85,12.32,0.0,389.8,0.0,1000.0,obsstart,351.9,389.8,364.7,227.2,unknown,0.9,20200226
70514,unknown,--,unknown,unknown,16,1680.0,132.85,12.32,0.0,118.3,0.0,1000.0,obsstart,112.0,118.3,120.2,74.4,unknown,0.9,20200227
70502,unknown,--,unknown,unknown,41,8415.0,180.0,-0.5,0.0,358.2,0.0,1000.0,obsstart,317.9,358.2,325.0,260.2,unknown,0.9,20200227
70513,unknown,--,unknown,unknown,13,2220.0,133.42,11.65,0.0,3.7,0.0,1000.0,obsstart,3.4,3.7,3.5,2.5,unknown,0.9,20200229
70500,unknown,--,unknown,unknown,14,7000.0,119.0,50.0,0.0,1389.0,0.0,1000.0,obsend,1145.4,1389.0,1160.5,770.4,unknown,0.9,20200303
70005,unknown,--,unknown,unknown,17,13500.0,158.0,25.0,0.0,8463.1,0.0,1000.0,obsend,8088.5,8463.1,8858.6,9174.7,unknown,0.9,20200303


In [5]:
tiles_table[tiles_table['TILEID'] == tileid]

TILEID,SURVEY,PROGRAM,FAPRGRM,FAFLAVOR,NEXP,EXPTIME,TILERA,TILEDEC,EFFTIME_ETC,EFFTIME_SPEC,EFFTIME_GFA,GOALTIME,OBSSTATUS,LRG_EFFTIME_DARK,ELG_EFFTIME_DARK,BGS_EFFTIME_BRIGHT,LYA_EFFTIME_DARK,GOALTYPE,MINTFRAC,LASTNIGHT
int64,str7,str6,str16,str19,int64,float64,float64,float64,float64,float64,float64,float64,str8,float64,float64,float64,float64,str7,float64,int64
3867,main,--,dark,maindark,1,1355.5,130.832,69.075,1008.4,1108.3,966.7,1000.0,obsend,1108.3,1191.1,1081.0,1111.7,dark,0.85,20211126


In [6]:
row_index = np.where(tiles_table['TILEID'] == tileid)[0]
assert len(row_index) == 1
load_tile = dict()
for c in db.Tile.__table__.columns:
    if c.name == 'program' and tiles_table[row_index[0]]['PROGRAM'].mask:
        load_tile[c.name] = 'dark'
    else:
        load_tile[c.name] = tiles_table[row_index[0]][c.name.upper()].tolist()
new_tile = db.Tile(**load_tile)

In [7]:
# db.dbSession.rollback()
db.dbSession.add(new_tile)
db.dbSession.commit()

## Read exposures file

In [9]:
exposures_file = findfile('exposures')
exposures_table = Table.read(exposures_file, format='fits', hdu='EXPOSURES')
frames_table = Table.read(exposures_file, format='fits', hdu='FRAMES')
exposures_table[exposures_table['TILEID'] == tileid]

NIGHT,EXPID,TILEID,TILERA,TILEDEC,MJD,SURVEY,PROGRAM,FAPRGRM,FAFLAVOR,EXPTIME,EFFTIME_SPEC,GOALTIME,GOALTYPE,MINTFRAC,AIRMASS,EBV,SEEING_ETC,EFFTIME_ETC,TSNR2_ELG,TSNR2_QSO,TSNR2_LRG,TSNR2_LYA,TSNR2_BGS,TSNR2_GPBDARK,TSNR2_GPBBRIGHT,TSNR2_GPBBACKUP,LRG_EFFTIME_DARK,ELG_EFFTIME_DARK,BGS_EFFTIME_BRIGHT,LYA_EFFTIME_DARK,GPB_EFFTIME_DARK,GPB_EFFTIME_BRIGHT,GPB_EFFTIME_BACKUP,TRANSPARENCY_GFA,SEEING_GFA,FIBER_FRACFLUX_GFA,FIBER_FRACFLUX_ELG_GFA,FIBER_FRACFLUX_BGS_GFA,FIBERFAC_GFA,FIBERFAC_ELG_GFA,FIBERFAC_BGS_GFA,AIRMASS_GFA,SKY_MAG_AB_GFA,SKY_MAG_G_SPEC,SKY_MAG_R_SPEC,SKY_MAG_Z_SPEC,EFFTIME_GFA,EFFTIME_DARK_GFA,EFFTIME_BRIGHT_GFA,EFFTIME_BACKUP_GFA
int32,int32,int32,float64,float64,float64,bytes7,bytes6,bytes19,bytes19,float64,float64,float64,bytes7,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64
20211126,110987,3867,130.832,69.075,59545.42359139,main,dark,dark,maindark,1355.54345703125,1108.266357421875,1000.0,dark,0.85,1.2937510013580322,0.0328538715839386,0.7816540002822876,1008.3909912109375,138.49758911132812,36.12108612060547,91.2107925415039,94.20612081214786,7721.27587890625,12680.3271484375,2221.425537109375,16633.6796875,1108.266357421875,1191.079345703125,1080.9786376953125,1111.71786024777,1063.00244140625,1162.42626953125,1178.088623046875,1.0194541169953857,0.784009214278641,0.774699396350431,0.5381829509439544,0.24051108697514237,1.2897602996518225,1.229704022020451,1.1984016686497583,1.2852353271869736,20.334894382343847,20.66803869771934,20.317365959665477,18.984880968726785,966.671222839215,966.671222839215,1050.159859268238,1518.3969558168317


In [10]:
row_index = np.where(exposures_table['TILEID'] == tileid)[0]
assert len(row_index) > 0
data_columns = list()
for c in db.Exposure.__table__.columns:
    if c.name == 'date_obs':
        data_columns.append(list(map(utc.localize, Time(exposures_table[row_index]['MJD'], format='mjd').to_value('datetime').tolist())))
    else:
        data_columns.append(exposures_table[row_index][c.name.upper()].tolist())
data_rows = list(zip(*data_columns))
load_exposures = [db.Exposure(**(dict([(c.name, d) for c, d in zip(db.Exposure.__table__.columns, row)]))) for row in data_rows]
# load_exposures

In [11]:
# db.dbSession.rollback()
db.dbSession.add_all(load_exposures)
db.dbSession.commit()

In [12]:
load_frames = list()
for exposure in load_exposures:
    row_index = np.where(frames_table['EXPID'] == exposure.expid)[0]
    assert len(row_index) > 0
    data_columns = list()
    for c in db.Frame.__table__.columns:
        if c.name == 'frameid':
            data_columns.append((100*frames_table[row_index]['EXPID'] + np.array([cameraid(c) for c in frames_table[row_index]['CAMERA']], dtype=frames_table[row_index]['EXPID'].dtype)).tolist())
        else:
            data_columns.append(frames_table[row_index][c.name.upper()].tolist())
    data_rows = list(zip(*data_columns))
    load_frames += [db.Frame(**(dict([(c.name, d) for c, d in zip(db.Frame.__table__.columns, row)]))) for row in data_rows]
# load_frames

In [13]:
# db.dbSession.rollback()
db.dbSession.add_all(load_frames)
db.dbSession.commit()

## Load photometry for the tile

When tractor photometry is written out by John Moustakas' VAC code, only objects with `brickname != ''` are written.

In [14]:
def no_sky(catalog):
    """Identify objects in `catalog` that are not sky targets.
    
    Parameters
    ----------
    catalog : :class:`~astropy.table.Table`
        Any Table containing a ``TARGETID`` column.
    
    Returns
    -------
    :class:`numpy.ndarray`
        The indexes of rows that are not sky targets.
    """
    _, _, _, _, sky, _ = decode_targetid(catalog['TARGETID'])
    return np.where((sky == 0) & (catalog['TARGETID'] > 0))[0]

In [15]:
fiberassign_file = findfile('fiberassignsvn', tile=tileid)
potential_targets_table = Table.read(fiberassign_file, format='fits', hdu='TARGETS')
no_sky_rows = no_sky(potential_targets_table)
potential_cat = Table()
potential_cat['TARGETID'] = potential_targets_table['TARGETID'][no_sky_rows]
potential_cat['TILEID'] = tileid
potential_cat['TARGET_RA'] = potential_targets_table['RA'][no_sky_rows]
potential_cat['TARGET_DEC'] = potential_targets_table['DEC'][no_sky_rows]
# potential_cat['PETAL_LOC'] = potential_targets_table['PETAL_LOC'][no_sky_rows]
potential_cat['SURVEY'] = new_tile.survey
potential_cat['PROGRAM'] = 'dark'
# potential_cat

In [16]:
potential_targetphot = gather_targetphot(potential_cat, racolumn='TARGET_RA', deccolumn='TARGET_DEC')
potential_targetphot['SURVEY'] = potential_cat['SURVEY']
potential_targetphot['PROGRAM'] = potential_cat['PROGRAM']
potential_targetphot['TILEID'] = potential_cat['TILEID']
potential_targetphot['DESINAME'] = radec_to_desiname(potential_targetphot['RA'], potential_targetphot['DEC'])
inan = np.logical_or(np.isnan(potential_targetphot['PMRA']), np.isnan(potential_targetphot['PMDEC']))
if np.any(inan):
    potential_targetphot['PMRA'][inan] = 0.0
    potential_targetphot['PMDEC'][inan] = 0.0
# potential_targetphot

In [17]:
potential_tractorphot = gather_tractorphot(potential_cat, racolumn='TARGET_RA', deccolumn='TARGET_DEC')

In [18]:
assert (np.where(potential_tractorphot['RELEASE'] == 0)[0] == np.where(potential_tractorphot['BRICKNAME'] == '')[0]).all()

In [19]:
row_index = np.where(potential_tractorphot['BRICKNAME'] != '')[0]
expand_dchisq = ('dchisq_psf', 'dchisq_rex', 'dchisq_dev', 'dchisq_exp', 'dchisq_ser',)
if len(row_index) > 0:
    data_columns = list()
    for c in db.Photometry.__table__.columns:
        if c.name == 'brick_objid':
            data_columns.append(potential_tractorphot[row_index]['OBJID'].tolist())
        elif c.name == 'morphtype':
            data_columns.append(potential_tractorphot[row_index]['TYPE'].tolist())
        elif c.name in expand_dchisq:
            j = expand_dchisq.index(c.name)
            data_columns.append(potential_tractorphot[row_index]['DCHISQ'][:, j].tolist())
        else:
            data_columns.append(potential_tractorphot[row_index][c.name.upper()].tolist())
data_rows = list(zip(*data_columns))
load_photometry = [db.Photometry(**(dict([(c.name, d) for c, d in zip(db.Photometry.__table__.columns, row)]))) for row in data_rows]
# load_photometry[:20]

In [20]:
# db.dbSession.rollback()
db.dbSession.add_all(load_photometry)
db.dbSession.commit()

### Load photometry, such as it is, for object that are not in the tractor catalog.

In [21]:
loaded_targetid = Table()
loaded_targetid['TARGETID'] = np.array([r.targetid for r in load_photometry])
loaded_targetid['LS_ID'] = np.array([r.ls_id for r in load_photometry])
j = join(potential_targetphot['TARGETID', 'RELEASE'], loaded_targetid, join_type='left', keys='TARGETID')
load_rows = np.zeros((len(potential_targetphot),), dtype=bool)
try:
    load_targetids = j['TARGETID'][j['LS_ID'].mask]
except AttributeError:
    #
    # This means *every* TARGETID is already loaded.
    #
    pass
else:
    unique_targetid, targetid_index = np.unique(potential_targetphot['TARGETID'].data, return_index=True)
    for t in load_targetids:
        load_rows[targetid_index[unique_targetid == t]] = True
# load_rows

In [22]:
row_index = load_rows
expand_dchisq = ('dchisq_psf', 'dchisq_rex', 'dchisq_dev', 'dchisq_exp', 'dchisq_ser',)
if len(row_index) > 0:
    data_columns = list()
    for c in db.Photometry.__table__.columns:
        if c.name == 'ls_id':
            ls_id = ((potential_targetphot['RELEASE'].data.astype(np.int64) << 40) |
                     (potential_targetphot['BRICKID'].data.astype(np.int64) << 16) |
                     potential_targetphot['BRICK_OBJID'].data.astype(np.int64))
            data_columns.append(ls_id.tolist())
        elif c.name in expand_dchisq:
            j = expand_dchisq.index(c.name)
            data_columns.append(potential_targetphot[row_index]['DCHISQ'][:, j].tolist())
        else:
            data_columns.append(potential_targetphot[row_index][c.name.upper()].tolist())
data_rows = list(zip(*data_columns))
load_targetphot = [db.Photometry(**(dict([(c.name, d) for c, d in zip(db.Photometry.__table__.columns, row)]))) for row in data_rows]
# load_targetphot[:20]

In [23]:
# db.dbSession.rollback()
db.dbSession.add_all(load_targetphot)
db.dbSession.commit()

In [24]:
row_index = np.arange(len(potential_targetphot))
if len(row_index) > 0:
    data_columns = list()
    for c in db.Target.__table__.columns:
        if c.name == 'id':
            s = np.array([surveyid(s) for s in potential_targetphot['SURVEY']], dtype=np.int64)
            id0 = s << 32 | potential_targetphot['TILEID'].base.astype(np.int64)
            data_columns.append([i0 << 64 | i1 for i0, i1 in zip(id0.tolist(), potential_targetphot['TARGETID'].tolist())])
        else:
            data_columns.append(potential_targetphot[row_index][c.name.upper()].tolist())
data_rows = list(zip(*data_columns))
load_target = [db.Target(**(dict([(c.name, d) for c, d in zip(db.Target.__table__.columns, row)]))) for row in data_rows]
# load_target[:20]

In [25]:
# db.dbSession.rollback()
db.dbSession.add_all(load_target)
db.dbSession.commit()

## Load tile/cumulative redshifts

Need a way to compute "best" spectra as new tiles are added. There are a lot of columns that come from other sources here, so need to track these down.

In [26]:
redrock_files = list()
for spectrograph in range(10):
    redrock_file, redrock_exists = findfile('redrock_tile', groupname='cumulative', tile=tileid, spectrograph=spectrograph, night=new_tile.lastnight, return_exists=True)
    if redrock_exists:
        redrock_files.append(redrock_file)

In [27]:
load_ztile = list()
for rr in redrock_files:
    redrock_table, expfibermap = read_redrock(rr, group='cumulative', recoadd_fibermap=True, pertile=True)
    firstnight = np.zeros(len(redrock_table), dtype=np.int32)
    row_index = no_sky(redrock_table)
    if len(row_index) > 0:
        data_columns = list()
        for c in db.Ztile.__table__.columns:
            if c.name == 'id':
                id0 = spgrpid('cumulative') << 27 | redrock_table['SPGRPVAL'][row_index].base.astype(np.int64)
                data_columns.append([(i0 << 64) | i1 for i0, i1 in zip(id0.tolist(), redrock_table['TARGETID'][row_index].tolist())])
            elif c.name == 'targetphotid':
                id0 = np.array([surveyid(new_tile.survey) << 32 | new_tile.tileid]*len(row_index), dtype=np.int64)
                data_columns.append([(i0 << 64) | i1 for i0, i1 in zip(id0.tolist(), redrock_table['TARGETID'][row_index].tolist())])
            elif c.name == 'spgrp':
                data_columns.append(['cumulative']*len(row_index))
            elif c.name == 'desiname':
                data_columns.append(radec_to_desiname(redrock_table['TARGET_RA'][row_index], redrock_table['TARGET_DEC'][row_index]).tolist())
            elif c.name == 'survey':
                data_columns.append([new_tile.survey]*len(row_index))
            elif c.name == 'program':
                data_columns.append([new_tile.program]*len(row_index))
            elif c.name == 'firstnight':
                for tilefm in Table(expfibermap[['TILEID', 'NIGHT']]).group_by('TILEID').groups:
                    tileid = tilefm['TILEID'][0]
                    iitile = redrock_table['TILEID'] == new_tile.tileid
                    firstnight[iitile] = np.min(tilefm['NIGHT'])
                assert (firstnight != 0).all()
                data_columns.append(firstnight[row_index].tolist())
            elif c.name in ('sv_nspec', 'main_nspec', 'zcat_nspec'):
                data_columns.append([0]*len(row_index))
            elif c.name in ('sv_primary', 'main_primary', 'zcat_primary'):
                data_columns.append([False]*len(row_index))
            # elif c.name in ('min_mjd', 'mean_mjd', 'max_mjd'):
            #     data_columns.append([0.0]*len(row_index))
            elif c.name.startswith('coeff_'):
                coeff_index = int(c.name.split('_')[1])
                data_columns.append(redrock_table[row_index]['COEFF'][:, coeff_index].tolist())
            else:
                data_columns.append(redrock_table[row_index][c.name.upper()].tolist())
    data_rows = list(zip(*data_columns))
    load_ztile += [db.Ztile(**(dict([(c.name, d) for c, d in zip(db.Ztile.__table__.columns, row)]))) for row in data_rows]

INFO:zcatalog.py:114:read_redrock: Reading /global/cfs/cdirs/desi/spectro/redux/daily/tiles/cumulative/3867/20211126/redrock-0-3867-thru20211126.fits
INFO:zcatalog.py:128:read_redrock: Recoadding fibermap from spectra-0-3867-thru20211126.fits
INFO:zcatalog.py:114:read_redrock: Reading /global/cfs/cdirs/desi/spectro/redux/daily/tiles/cumulative/3867/20211126/redrock-1-3867-thru20211126.fits
INFO:zcatalog.py:128:read_redrock: Recoadding fibermap from spectra-1-3867-thru20211126.fits
INFO:zcatalog.py:114:read_redrock: Reading /global/cfs/cdirs/desi/spectro/redux/daily/tiles/cumulative/3867/20211126/redrock-2-3867-thru20211126.fits
INFO:zcatalog.py:128:read_redrock: Recoadding fibermap from spectra-2-3867-thru20211126.fits
INFO:zcatalog.py:114:read_redrock: Reading /global/cfs/cdirs/desi/spectro/redux/daily/tiles/cumulative/3867/20211126/redrock-3-3867-thru20211126.fits
INFO:zcatalog.py:128:read_redrock: Recoadding fibermap from spectra-3-3867-thru20211126.fits
INFO:zcatalog.py:114:read_re

In [28]:
# db.dbSession.rollback()
db.dbSession.add_all(load_ztile)
db.dbSession.commit()

## Load fiberassign and potential

In [29]:
fiberassign_table = Table.read(fiberassign_file, format='fits', hdu='FIBERASSIGN')
potential_table = Table.read(fiberassign_file, format='fits', hdu='POTENTIAL_ASSIGNMENTS')

In [30]:
row_index = no_sky(fiberassign_table)
if len(row_index) > 0:
    data_columns = list()
    for c in db.Fiberassign.__table__.columns:
        if c.name == 'id':
            id0 = (fiberassign_table['LOCATION'][row_index].base.astype(np.int64) << 32) | new_tile.tileid
            data_columns.append([(i0 << 64) | i1 for i0, i1 in zip(id0.tolist(), fiberassign_table['TARGETID'][row_index].tolist())])
        elif c.name == 'tileid':
            data_columns.append([new_tile.tileid]*len(row_index))
        else:
            data_columns.append(fiberassign_table[c.name.upper()][row_index].tolist())
data_rows = list(zip(*data_columns))
load_fiberassign = [db.Fiberassign(**(dict([(c.name, d) for c, d in zip(db.Fiberassign.__table__.columns, row)]))) for row in data_rows]
# load_fiberassign

[Fiberassign(tileid=3867, targetid=39633471919883859, location=311),
 Fiberassign(tileid=3867, targetid=39633471919884932, location=272),
 Fiberassign(tileid=3867, targetid=39633471924077291, location=252),
 Fiberassign(tileid=3867, targetid=39633474197391529, location=156),
 Fiberassign(tileid=3867, targetid=39633471924076786, location=204),
 Fiberassign(tileid=3867, targetid=39633471924078031, location=233),
 Fiberassign(tileid=3867, targetid=39633474197390347, location=172),
 Fiberassign(tileid=3867, targetid=39633471919884249, location=310),
 Fiberassign(tileid=3867, targetid=39633471919885170, location=290),
 Fiberassign(tileid=3867, targetid=39633471924078665, location=216),
 Fiberassign(tileid=3867, targetid=39633471919885225, location=237),
 Fiberassign(tileid=3867, targetid=39633474197391307, location=170),
 Fiberassign(tileid=3867, targetid=39633471924078403, location=186),
 Fiberassign(tileid=3867, targetid=39633471924078034, location=217),
 Fiberassign(tileid=3867, targetid

In [31]:
# db.dbSession.rollback()
db.dbSession.add_all(load_fiberassign)
db.dbSession.commit()

In [32]:
row_index = no_sky(potential_table)
if len(row_index) > 0:
    data_columns = list()
    for c in db.Potential.__table__.columns:
        if c.name == 'id':
            id0 = (potential_table['LOCATION'][row_index].base.astype(np.int64) << 32) | new_tile.tileid
            data_columns.append([(i0 << 64) | i1 for i0, i1 in zip(id0.tolist(), potential_table['TARGETID'][row_index].tolist())])
        elif c.name == 'tileid':
            data_columns.append([new_tile.tileid]*len(row_index))
        else:
            data_columns.append(potential_table[c.name.upper()][row_index].tolist())
data_rows = list(zip(*data_columns))
load_potential = [db.Potential(**(dict([(c.name, d) for c, d in zip(db.Potential.__table__.columns, row)]))) for row in data_rows]
# load_potential

[Potential(tileid=3867, targetid=39633480866333157, location=0),
 Potential(tileid=3867, targetid=39633480866333134, location=0),
 Potential(tileid=3867, targetid=39633480866333114, location=0),
 Potential(tileid=3867, targetid=39633480866333080, location=0),
 Potential(tileid=3867, targetid=39633480866332927, location=0),
 Potential(tileid=3867, targetid=39633480866332908, location=0),
 Potential(tileid=3867, targetid=39633480862141737, location=0),
 Potential(tileid=3867, targetid=39633480862141720, location=0),
 Potential(tileid=3867, targetid=39633480866333248, location=1),
 Potential(tileid=3867, targetid=39633480866333319, location=1),
 Potential(tileid=3867, targetid=39633480866332962, location=1),
 Potential(tileid=3867, targetid=39633480866332937, location=1),
 Potential(tileid=3867, targetid=39633480862141579, location=2),
 Potential(tileid=3867, targetid=39633480862141538, location=2),
 Potential(tileid=3867, targetid=39633480862141544, location=2),
 Potential(tileid=3867, t

In [33]:
# db.dbSession.rollback()
db.dbSession.add_all(load_potential)
db.dbSession.commit()

## Recompute global values

The global values are the primary classification and number of spectra.

In [35]:
zall_tilecumulative = db.dbSession.query(db.Ztile).all()
zall_table = Table(list(zip(*[(z.targetid, z.zwarn, z.tsnr2_lrg) for z in zall_tilecumulative])), names=('TARGETID', 'ZWARN', 'TSNR2_LRG'))
nspec, primary = find_primary_spectra(zall_table)
zcat_nspec, zcat_primary = nspec.tolist(), primary.tolist()
for k, z in enumerate(zall_tilecumulative):
    z.zcat_nspec = zcat_nspec[k]
    z.zcat_primary = zcat_primary[k]
db.dbSession.commit()

In [36]:
sv_tilecumulative = db.dbSession.query(db.Ztile).filter(db.Ztile.survey.in_(('sv1', 'sv2', 'sv3'))).all()
if len(sv_tilecumulative) > 0:
    sv_table = Table(list(zip(*[(z.targetid, z.zwarn, z.tsnr2_lrg) for z in sv_tilecumulative])), names=('TARGETID', 'ZWARN', 'TSNR2_LRG'))
    nspec, primary = find_primary_spectra(sv_table)
    sv_nspec, sv_primary = nspec.tolist(), primary.tolist()
    for k, z in enumerate(sv_tilecumulative):
        z.sv_nspec = sv_nspec[k]
        z.sv_primary = sv_primary[k]
    db.dbSession.commit()

In [37]:
main_tilecumulative = db.dbSession.query(db.Ztile).filter(db.Ztile.survey.in_(('main', ))).all()
if len(main_tilecumulative) > 0:
    main_table = Table(list(zip(*[(z.targetid, z.zwarn, z.tsnr2_lrg) for z in main_tilecumulative])), names=('TARGETID', 'ZWARN', 'TSNR2_LRG'))
    nspec, primary = find_primary_spectra(main_table)
    main_nspec, main_primary = nspec.tolist(), primary.tolist()
    for k, z in enumerate(main_tilecumulative):
        z.main_nspec = main_nspec[k]
        z.main_primary = main_primary[k]
    db.dbSession.commit()